# 🚦 Predicción de Tráfico con Redes Neuronales LSTM
Este cuaderno aplica un modelo LSTM (Long Short-Term Memory) para predecir el conteo de vehículos a partir de datos históricos por hora en un segmento vial.

## 📚 Paso 1: Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

## 📂 Paso 2: Cargar archivo de datos de tráfico (CSV)

In [ ]:
from google.colab import files
import io

uploaded = files.upload()
for filename in uploaded:
    df = pd.read_csv(io.BytesIO(uploaded[filename]), parse_dates=['fecha_hora'])
    break
df.head()

## 📊 Paso 3: Visualización del conteo de vehículos

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(df['fecha_hora'], df['conteo_vehiculos'], label='Conteo de vehículos')
plt.xlabel('Fecha y hora')
plt.ylabel('Número de vehículos')
plt.title('Tráfico por hora')
plt.grid(True)
plt.tight_layout()
plt.show()

## ⚙️ Paso 4: Preparar datos para LSTM

In [ ]:
# Reordenar y escalar
data = df[['conteo_vehiculos']].values
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Crear secuencias para LSTM
def crear_secuencias(datos, pasos):
    X, y = [], []
    for i in range(pasos, len(datos)):
        X.append(datos[i-pasos:i, 0])
        y.append(datos[i, 0])
    return np.array(X), np.array(y)

pasos = 24  # predicción con ventana de 24 horas
X, y = crear_secuencias(scaled_data, pasos)
X = X.reshape((X.shape[0], X.shape[1], 1))

## 🧠 Paso 5: Construir y entrenar el modelo LSTM

In [ ]:
# Dividir entrenamiento y prueba
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Definir modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(pasos, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Entrenamiento
es = EarlyStopping(patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[es], verbose=0)

## 📈 Paso 6: Visualizar predicciones

In [ ]:
# Predicción
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Graficar
plt.figure(figsize=(12,5))
plt.plot(y_test_inv, label='Real')
plt.plot(y_pred_inv, label='Predicción')
plt.title('Predicción de tráfico (últimos datos)')
plt.xlabel('Horas')
plt.ylabel('Conteo de vehículos')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()